In [1]:
!git clone https://github.com/1stDayHack/FDK.git
!pip install detectron2 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
!pip install streamlit -q
!pip install -r /content/FDK/requirements.txt

Cloning into 'FDK'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 1939 (delta 21), reused 22 (delta 4), pack-reused 1892
Receiving objects: 100% (1939/1939), 487.14 MiB | 42.98 MiB/s, done.
Resolving deltas: 100% (217/217), done.
Checking out files: 100% (1600/1600), done.
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.7/index.html
     |████████████████████████████████| 6.8MB 610kB/s 
     |████████████████████████████████| 2.2MB 21.0MB/s 
     |████████████████████████████████| 276kB 49.0MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.2.post20201213-cp36-none-any.whl size=40711 sha256=d5e31bfcb1301147987f4cf6b304961d6d41acf76575e46aeca4b9dea7d4ade8
  Stored in directory: /root/.cache/pip/wheels/20/eb/f9/b537cf42e3b4c57fe2a49aecad264c775df2d89e0ffbf5b17d
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44620 sh

     |████████████████████████████████| 7.4MB 10.8MB/s 
     |████████████████████████████████| 102kB 14.0MB/s 
     |████████████████████████████████| 163kB 54.0MB/s 
     |████████████████████████████████| 112kB 59.7MB/s 
     |████████████████████████████████| 4.5MB 49.6MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 122kB 61.7MB/s 
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.4.2 which is incompatible.
     |████████████████████████████████| 7.6MB 11.8MB/s 
     |████████████████████████████████| 11.6MB 49.2MB/s 
     |████████████████████████████████| 49.3MB 145kB/s 
     |████████████████████████████████| 12.4MB 260kB/s 
     |████████████████████████████████| 778kB 55.0MB/s 
     |████████████████████████████████| 2.1MB 46.4MB/s 
     |████████████████████████████████| 3.0MB 51.6MB/s 
     |████████████████████████████████| 1.1MB 55.9MB/s 
     |████████████████████████████████| 890kB 

In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -qq ngrok-stable-linux-amd64.zip

--2020-12-13 02:31:53--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.196.106.64, 3.213.223.141, 52.22.239.120, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.196.106.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  46.9MB/s    in 0.3s    

2020-12-13 02:31:53 (46.9 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]



In [3]:
get_ipython().system_raw('./ngrok http 8501 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://0281d5753099.ngrok.io


In [4]:
%%writefile streamlights.py
import streamlit as st
from time import time
from time import sleep
from torch.cuda import is_available
import os
import cv2
import PIL
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from bokeh.plotting import figure
from FDK.src.core.detect import Detector
from FDK.src.core.utils.utils import cv2_to_pil


def video_to_frames(file_name: str, step=1, frames=1000) -> list:
    global vidcap
    vidcap = cv2.VideoCapture(file_name)
    success, image = vidcap.read()
    image_list = []

    while success:
        image_list.append(image)

        if len(image_list) >= frames:
            break
        
        for _ in range(step):
            success, image = vidcap.read()
  
    return image_list


def frames_to_video(frames: list, filename="output", fps=5, codec="DIVX") -> cv2.VideoWriter:
    video = cv2.VideoWriter(filename + ".mp4", cv2.VideoWriter_fourcc(*codec), fps, frames[0].shape[0:2][::-1])

    for frame in frames:
        video.write(frame)
  
    return video

CAR_TAG = 2

class CountPredictions:
    def __init__(self, frames: list, entry_rect: tuple, light_states={}, min_score=0.8, visualise=False, figsize=(10, 10)):
        self.count = 0
        self.frames = frames
        self.redlight_counts = []
        self.greenlight_counts = []
        self.counts = []
        self.light_states = light_states
        self.visualise = visualise
        self.visualised_frames = []
        self.figsize = figsize
        self.min_score = min_score
        self.entry_rect = entry_rect
  
    def process_frame(self) -> float:
        self.greenlight_counts = []
        different_light_states = len(self.light_states) > 0
        frame_count = len(self.frames)

        if different_light_states:
            was_green = list(self.light_states.values())[0]
    
        else:
            was_green = True

        if self.visualise:
            self.visualised_frames = []

        for i, frame in enumerate(self.frames):
            output = detector.predict(frame)
            instance = output["instances"]

            if self.visualise:
                self.visualised_frames.append(detector.visualize(frame, output, figsize=self.figsize, noplot=True))

            if different_light_states:
                is_green = _locate_index_in_dict(self.light_states, i)
      
            else:
                is_green = True

            if i == 0:
                self.greenlight_counts.append(count_cars_in_image(instance, self.min_score))

            elif is_green:
                if was_green:
                    self.greenlight_counts.append(count_cars_in_image(instance, self.min_score))
                    #self.greenlight_counts.append(_count_boxes_in_region([instance.pred_boxes[i] for i, tag in enumerate(instance.pred_classes) if tag == CAR_TAG and instance.scores[i] >= self.min_score], self.entry_rect[0], self.entry_rect[1]))
      
                else:
                    self.count += count_cars_in_image(instance, self.min_score)
            else:
                self.redlight_counts.append(count_cars_in_image(instance, self.min_score))

            self.counts.append(count_cars_in_image(instance, self.min_score))


            was_green = is_green
            yield i / frame_count
  
        last_x = 0
        for x in self.greenlight_counts:
            change = x - last_x
            if change > 0:
                self.count += change
            last_x = x
    
        return 1.0

def is_car_in_image(instance, min_score=0.8) -> bool:
    for i, tag in instance.pred_classes:
        if tag == CAR_TAG and instance.pred_scores[i] >= min_score:
            return True

    return False


def count_cars_in_image(instance, min_score=0.8) -> int:
    count = 0
    for i, tag in enumerate(instance.pred_classes):
        if tag == CAR_TAG and instance.scores[i] >= min_score:
            count += 1
    
    return count


def decreased_index(counts: list) -> list:
    decreased = []
    last_x = 0
    for i, x in enumerate(counts):
        if x - last_x < 0:
            decreased.append(i)
        last_x = x
    return decreased


def _locate_index_in_dict(dictionary: dict, index: int):
    # if a dictionary is in the form {1: obj1, 6: obj2, 13: obj3 etc}
    # this function will find the element which the index fits in
    # so an index of 2 will return obj1 while 15 will return obj3, and 7 will return obj2
    for idx in dictionary:
        if index >= idx:
            return dictionary[idx]


# count cars
def _count_boxes_in_region(boxes: list, top_left: tuple, bottom_right: tuple) -> int:
    # Each box is in the format [x1, y1, x2, y2] probably
    count = 0
    for i, box in enumerate(boxes):
        tensor = box.tensor[0]
        if (top_left[0] <= float(tensor[0]) <= bottom_right[0] and top_left[1] <= float(tensor[1]) <= bottom_right[1]) or (top_left[0] <= float(tensor[2]) <= bottom_right[0] and top_left[1] <= float(tensor[3]) <= bottom_right[1]):
            count += 1

    return count

#Streamlit


#Head

st.set_page_config(
    page_title='Streamlights',
    page_icon=':vertical_traffic_light:',
    layout='centered',
    initial_sidebar_state='expanded'
)

max_width = 80
st.markdown(
        f"""
<style>
    .reportview-container .main .block-container{{
        max-width: {max_width}%;
    }}
</style>
""",
        unsafe_allow_html=True,
    )


st.title('Welcome to Streamlights')
st.markdown('''Streamlights, a smart traffic light program that scans all incoming cars on the road and measures real-time traffic flow.<br/>
Make valuable traffic data available for government road and public transportation planning.<br/>
Information on red light runners made available for government action.<br/>
When roads are empty, actuated signals can be sent to prolong green light signals at other roads in order to reduce vehicle idle time.''', unsafe_allow_html=True)

#Body

#Display instructions in sidebar
instructions = st.sidebar.beta_expander('Instructions')
instructions.subheader('Step 1.')
instructions.write('Upload a video for 2 roads of the same junction each. Ensure that the video formats are either .mp4 or .avi')
instructions.subheader('Step 2.')
instructions.write('''When the videos are done uploading, select an fps ratio (output fps/original fps) and process the videos.
The videos will be processed with Detectron2, and the number of cars per frame will be returned.
This process may take awhile. To reduce processing time, run Google Colaboratory with a GPU hardware accelerator.''')
instructions.subheader('Step 3.')
instructions.write('View output videos and statistics')


uploader = st.beta_expander('Upload Videos')
uploader1 = uploader.beta_container()
uploader2 = uploader.beta_container()

video1 = uploader1.file_uploader('Upload a video of road 1')
if video1 is not None:
    try:
        if video1.name.endswith('.mp4') or video1.endswith('.avi'):
            uploader1.success('Uploaded')
            open(video1.name, 'wb').write(video1.getvalue())
            uploader1.video(video1.name)
        else:
            uploader1.error('File not supported, please use either .avi or .mp4 files instead')
            video1 = None
    except AttributeError:
        uploader1.error('File not supported, please use either .avi or .mp4 files instead')

video2 = uploader2.file_uploader('Upload a video of road 2')
if video2 is not None:
    try:
        if video2.name.endswith('.mp4') or video2.endswith('.avi'):
            uploader2.success('Uploaded')
            open(video2.name, 'wb').write(video2.getvalue())
            uploader2.video(video2.name)
        else:
            uploader2.error('File not supported, please use either .avi or .mp4 files instead')
            video2 = None
    except AttributeError:
        uploader2.error('File not supported, please use either .avi or .mp4 files instead')


if video1 is not None and video2 is not None:

    if is_available():
        device = uploader.selectbox('Select device to process videos', ('CPU', 'GPU'), index=1)
        if device == 'CPU':
            device = 'cpu'
        else:
            device = 'cuda'
    else:
        device = 'cpu'

    # Instantiate detector
    detector = Detector(name="MyDet", device=device)

    #fps ratio slider
    fps_ratio = uploader.slider('Select fps ratio', min_value=0.1, max_value=1.0)
    uploader.write('Note: a greater fps ratio would yield smoother videos, but would take longer to process')

    if uploader.button('Process videos'):

        (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

        #Process video1
        video1_frames = video_to_frames(video1.name, int(1 / fps_ratio))
        if int(major_ver)  < 3 :
            fps1 = vidcap.get(cv2.cv.CV_CAP_PROP_FPS)
        else :
            fps1 = vidcap.get(cv2.CAP_PROP_FPS)
        video1_counter = CountPredictions(video1_frames, ((0, 0), video1_frames[0].shape[0:2][::-1]), light_states={0: False}, visualise=True)

        #Progress bar
        t = time()
        progress_container = uploader.empty()
        progress_container.write('Processing video 1...')
        progress_bar = uploader.empty()
        bar1 = progress_bar.progress(0)
        secs_left = uploader.empty()
        for progress in video1_counter.process_frame():
            if progress > 0:
                bar1.progress(progress)
                seconds = int((time() - t) * (1 / progress - 1))
                if seconds == 1:
                    secs_left.write(str(seconds) + " second remaining")
                else:
                    secs_left.write(str(seconds) + " seconds remaining")

        #Process video2
        video2_frames = video_to_frames(video2.name, int(1 / fps_ratio))
        if int(major_ver)  < 3 :
            fps2 = vidcap.get(cv2.cv.CV_CAP_PROP_FPS)
        else :
            fps2 = vidcap.get(cv2.CAP_PROP_FPS)
        video2_counter = CountPredictions(video2_frames, ((0, 0), video2_frames[0].shape[0:2][::-1]), light_states={0: False}, visualise=True)

        #Progress bar
        t = time()
        progress_container.write('Processing video 2...')
        bar2 = progress_bar.progress(0)
        for progress in video2_counter.process_frame():
            if progress > 0:
                bar2.progress(progress)
                seconds = int((time() - t) * (1 / progress - 1))
                if seconds == 1:
                    secs_left.write(str(seconds) + " second remaining")
                elif seconds == 0:
                    secs_left.write("Generating data...")
                else:
                    secs_left.write(str(seconds) + " seconds remaining")
        progress_container.empty()
        secs_left.empty()
        progress_bar.empty()

        #Convert processed frames to video
        video1_processed = frames_to_video(video1_counter.visualised_frames, filename='video1_processed', fps=fps1 * fps_ratio, codec='DIVX')
        video2_processed = frames_to_video(video2_counter.visualised_frames, filename='video2_processed', fps=fps2 * fps_ratio, codec='DIVX')
        video1_processed.release()
        video2_processed.release()

        #Convert video fourcc codec to X264
        os.system('ffmpeg -y -i video1_processed.mp4 -vcodec libx264 output1.mp4')
        os.system('ffmpeg -y -i video2_processed.mp4 -vcodec libx264 output2.mp4')


        #Display videos
        video_displayer = st.beta_expander('Output videos')

        video_displayer.subheader('Road 1')
        video_displayer.video('output1.mp4')

        video_displayer.subheader('Road 2')
        video_displayer.video('output2.mp4')

        #Display red light runners
        red_runners = st.beta_expander('Red light runners')

        red_runners.markdown('For demonstration purposes, we assume that the traffic lights are red for the entire duration of the video for both roads.')
        red_runners.markdown('In the future, this program can be synchronized with the traffic light states, as well as recognize and record the car plates of red light runners')
        red_runners.markdown('As of now, the right light runners are the cars at the bottommost of each image.')

        runners1 = (decreased_index(video1_counter.redlight_counts))
        runners2 = (decreased_index(video2_counter.redlight_counts))
        road1_runners = red_runners.beta_container()
        road2_runners = red_runners.beta_container()

        road1_runners.subheader('Road 1')
        road1_col1, road1_col2, road1_col3, road1_col4 = road1_runners.beta_columns(4)
        road1_column_list = [road1_col1, road1_col2, road1_col3, road1_col4]
        for i, index in enumerate(runners1):
            road1_column_list[i % 4].image(video1_counter.visualised_frames[index], use_column_width=True)

        road2_runners.subheader('Road 2')
        road2_col1, road2_col2, road2_col3, road2_col4 = road2_runners.beta_columns(4)
        road2_column_list = [road2_col1, road2_col2, road2_col3, road2_col4]
        for i, index in enumerate(runners2):
            road2_column_list[i % 4].image(video2_counter.visualised_frames[index], use_column_width=True)

        #Display data and statistics
        data_section = st.beta_expander('Data and Statistics')
        
        x1 = []
        x2 = []
        y1 = video1_counter.counts
        y2 = video2_counter.counts
        for i in range(len(video1_counter.counts)):
            x1.append(i / (fps1 * fps_ratio))
        for i in range(len(video2_counter.counts)):
            x2.append(i / (fps2 * fps_ratio))

        p = figure(title='Graph of amount of cars in frame against time', x_axis_label='Time / seconds', y_axis_label='Amount of cars in frame')
        p.line(x1, y1, legend='Road 1', line_width=2, line_color='red')
        p.line(x2, y2, legend='Road 2', line_width=2, line_color='blue')
        data_section.bokeh_chart(p, use_container_width=True)

        data_section.markdown('Average number of cars per frame for road 1: ' + '**' + str(round(sum(video1_counter.counts) / len(video1_counter.counts), 1)) + '**')
        data_section.markdown('Average number of cars per frame for road 2: ' + '**' + str(round(sum(video2_counter.counts) / len(video2_counter.counts), 1)) + '**')


Writing streamlights.py


In [5]:
!streamlit run streamlights.py


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.245.13.122:8501

2020-12-13 02:32:26.267331: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-12-13 02:32:27.652 ** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

2020-12-13 02:32:27.653 ** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 

2020-12-13 02:36:37.135 Loading checkpoint from https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl
2020-12-13 02:36:37.136 Downloading https://dl.fbaipublicfiles.com/detectron2/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl ...
2020-12-13 02:36:3